TEXTCNN文本分类

In [1]:
# 显卡查看
! nvidia-smi

Sun Jul  5 02:53:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    29W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 依赖安装
! pip install fastNLP

加载数据集

In [3]:
import sys

from fastNLP.core import Const

sys.path.insert(0, '/content/drive/My Drive/my_framework/qyt_clue/')  # 定义搜索路径的优先顺序，序号从0开始，表示最大优先级

import myClue  # noqa
print('myClue module path :{}'.format(myClue.__file__))  # 输出测试模块文件位置
from myClue.core import logger  # noqa
from myClue.core.utils import print_data_bundle  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.loader.classification import THUCNewsLoader  # noqa
from myClue.pipe.classification import THUCNewsPipe  # noqa

myClue module path :/content/drive/My Drive/my_framework/qyt_clue/myClue/__init__.py


In [4]:
import codecs
from myClue.tools.file import init_file_path  # noqa

In [5]:
# 数据集去除空格处理
train_file_config = {
        'train': '/content/drive/My Drive/game/UCAS_NLP_TC/data_02_chars/traindata.txt',
        'dev': '/content/drive/My Drive/game/UCAS_NLP_TC/data_02_chars/devdata.txt',
    }

In [6]:
logger.info('数据加载')
data_loader = THUCNewsLoader()
data_bundle = data_loader.load(train_file_config)
print_data_bundle(data_bundle)
logger.info('数据预处理')
data_pipe = THUCNewsPipe(tokenizer='white_space')
data_bundle = data_pipe.process(data_bundle)
data_bundle.rename_field(field_name=Const.CHAR_INPUT, new_field_name=Const.INPUT, ignore_miss_dataset=True, rename_vocab=True)
print_data_bundle(data_bundle)

2020-07-05 02:53:33 I [<ipython-input-6-d23eb2358db5>:1] 数据加载
2020-07-05 02:53:34 I [utils.py:16] dataset name : train
2020-07-05 02:53:34 I [utils.py:17] dataset len : 2500
2020-07-05 02:53:34 I [utils.py:18] dataset example : 
2020-07-05 02:53:34 I [utils.py:19] 
+-------------------------------------------+-------------+
| raw_chars                                 | target      |
+-------------------------------------------+-------------+
| 近 日 ， 胡 杏 儿 以 品 牌 挚 友 及 公... | EnterSports |
| 虎 扑 1 月 1 4 日 讯 根 据 《 世 界 体... | EnterSports |
| 日 前 ， 有 媒 体 爆 料 美 团 正 在 招... | Economics   |
| 记 者 1 日 从 天 津 大 学 获 悉 ， 该...  | Technology  |
| 贯 彻 落 实 党 的 十 九 大 的 战 略 部... | Military    |
+-------------------------------------------+-------------+
2020-07-05 02:53:34 I [utils.py:20] dataset 输出各个field的被设置成input和target的情况 : 
2020-07-05 02:53:34 I [utils.py:21] 
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | Fal

+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+


2020-07-05 02:53:45 I [utils.py:16] dataset name : train
2020-07-05 02:53:45 I [utils.py:17] dataset len : 2500
2020-07-05 02:53:45 I [utils.py:18] dataset example : 
2020-07-05 02:53:45 I [utils.py:19] 
+---------------------+--------+---------------------+---------+--------------------+
| raw_chars           | target | chars               | seq_len | words              |
+---------------------+--------+---------------------+---------+--------------------+
| 近 日 ， 胡 杏 ...  | 0      | ['近', '日', '，... | 155     | [250, 43, 2, 12... |
| 虎 扑 1 月 1 4 ...  | 0      | ['虎', '扑', '1...  | 230     | [1389, 1565, 8,... |
| 日 前 ， 有 媒 ...  | 1      | ['日', '前', '，... | 289     | [43, 65, 2, 14,... |
| 记 者 1 日 从 天... | 2      | ['记', '者', '1...  | 577     | [225, 92, 8, 43... |
| 贯 彻 落 实 党 ...  | 3      | ['贯', '彻', '落... | 1055    | [1261, 1176, 53... |
+---------------------+--------+---------------------+---------+--------------------+
2020-07-05 02:53:45 I [utils.py:20] dataset 输出各个field的被设置成

+-------------+-----------+--------+-----------+---------+-------+
| field_names | raw_chars | target | raw_words | seq_len | words |
+-------------+-----------+--------+-----------+---------+-------+
|   is_input  |   False   | False  |   False   |   True  |  True |
|  is_target  |   False   |  True  |   False   |  False  | False |
| ignore_type |           | False  |           |  False  | False |
|  pad_value  |           |   0    |           |    0    |   0   |
+-------------+-----------+--------+-----------+---------+-------+
+-------------+-----------+--------+-----------+---------+-------+
| field_names | raw_chars | target | raw_words | seq_len | words |
+-------------+-----------+--------+-----------+---------+-------+
|   is_input  |   False   | False  |   False   |   True  |  True |
|  is_target  |   False   |  True  |   False   |  False  | False |
| ignore_type |           | False  |           |  False  | False |
|  pad_value  |           |   0    |           |    0    |   0

In [7]:
import os
import sys
import time

import torch

from fastNLP import Const
from fastNLP.embeddings import StaticEmbedding
from fastNLP.models import CNNText
from fastNLP import Trainer
from fastNLP import CrossEntropyLoss
from torch.optim import Adam
from fastNLP import Tester

from myClue.core.metrics import ClassifyFPreRecMetric  # noqa
from myClue.core.callback import EarlyStopCallback  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.tools.file import init_file_path  # noqa

In [8]:
model_path = '/content/drive/My Drive/game/UCAS_NLP_TC/models/02_train_textcnn_chars'
init_file_path(model_path)
logger.add_file_handler(os.path.join(model_path, 'log_{}.txt'.format(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))))  # 日志写入文件
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
logger.warn('获取词典')
char_vocab = data_bundle.get_vocab('words')
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = data_bundle.get_vocab('target')
logger.info('target_vocab:{}'.format(target_vocab))
save_serialize_obj(char_vocab, char_vocab_pkl_file)
save_serialize_obj(target_vocab, target_vocab_pkl_file)
logger.info('词典序列化:{}'.format(char_vocab_pkl_file))
logger.warn('选择预训练词向量')
word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name='cn-char-fastnlp-100d')
logger.warn('神经网络模型')
model = CNNText(word2vec_embed, num_classes=len(target_vocab))
logger.info(model)
logger.warn('训练超参数设定')
loss = CrossEntropyLoss()
optimizer = Adam([param for param in model.parameters() if param.requires_grad])
# metric = AccuracyMetric()
metric = ClassifyFPreRecMetric(tag_vocab=data_bundle.get_vocab(Const.TARGET), only_gross=False)  # 若only_gross=False, 即还会返回各个label的metric统计值
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
logger.info('device:{}'.format(device))
batch_size = 32
n_epochs = 200
early_stopping = 10
trainer = Trainer(
    save_path=model_path,
    train_data=data_bundle.get_dataset('train'),
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epochs=n_epochs,
    dev_data=data_bundle.get_dataset('dev'),
    metrics=metric,
    metric_key='f',
    device=device,
    callbacks=[EarlyStopCallback(early_stopping)])
logger.warn('开始训练')
trainer.train()

2020-07-05 02:53:45 W [<ipython-input-8-c488b24f8c8b>:6] 获取词典
2020-07-05 02:53:45 I [<ipython-input-8-c488b24f8c8b>:8] char_vocab:Vocabulary(['近', '日', '，', '胡', '杏']...)
2020-07-05 02:53:45 I [<ipython-input-8-c488b24f8c8b>:10] target_vocab:Vocabulary(['EnterSports', 'Economics', 'Technology', 'Military', 'Government']...)
2020-07-05 02:53:45 I [<ipython-input-8-c488b24f8c8b>:13] 词典序列化:/content/drive/My Drive/game/UCAS_NLP_TC/models/02_train_textcnn_chars/vocab_char.pkl
2020-07-05 02:53:45 W [<ipython-input-8-c488b24f8c8b>:14] 选择预训练词向量
/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
2020-07-05 02:53:46 W [<ipython-input-8-c488b24f8c8b>:16] 神经网

Found 4985 out of 5033 words in the pre-training embedding.


2020-07-05 02:53:49 W [<ipython-input-8-c488b24f8c8b>:42] 开始训练


input fields after batch(if batch size is 2):
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 230]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2020-07-05-02-53-49-113738


2020-07-05 02:53:54 W [callback.py:38] ======epoch : 1 , early stopping : 0/10======
2020-07-05 02:53:54 I [callback.py:40] metric_key : f, metric_value : 0.636
2020-07-05 02:53:54 I [callback.py:41] eval_result : 
{'ClassifyFPreRecMetric': {'f-EnterSports': 0.804124, 'pre-EnterSports': 0.78, 'rec-EnterSports': 0.829787, 'f-Economics': 0.649573, 'pre-Economics': 0.76, 'rec-Economics': 0.567164, 'f-Technology': 0.578947, 'pre-Technology': 0.55, 'rec-Technology': 0.611111, 'f-Military': 0.663594, 'pre-Military': 0.72, 'rec-Military': 0.615385, 'f-Government': 0.448485, 'pre-Government': 0.37, 'rec-Government': 0.569231, 'f': 0.636, 'pre': 0.636, 'rec': 0.636}}


...

2020-07-05 02:55:40 W [callback.py:38] ======epoch : 20 , early stopping : 9/10======
2020-07-05 02:55:40 I [callback.py:40] metric_key : f, metric_value : 0.808
2020-07-05 02:55:40 I [callback.py:41] eval_result : 
{'ClassifyFPreRecMetric': {'f-EnterSports': 0.95, 'pre-EnterSports': 0.95, 'rec-EnterSports': 0.95, 'f-Economics': 0.747253, 'pre-Economics': 0.68, 'rec-Economics': 0.829268, 'f-Technology': 0.798206, 'pre-Technology': 0.89, 'rec-Technology': 0.723577, 'f-Military': 0.84264, 'pre-Military': 0.83, 'rec-Military': 0.85567, 'f-Government': 0.69697, 'pre-Government': 0.69, 'rec-Government': 0.704082, 'f': 0.808, 'pre': 0.808, 'rec': 0.808}}


Evaluate data in 0.46 seconds!
Evaluation on dev at Epoch 20/200. Step:1580/15800: 
ClassifyFPreRecMetric: f-EnterSports=0.95, pre-EnterSports=0.95, rec-EnterSports=0.95, f-Economics=0.747253, pre-Economics=0.68, rec-Economics=0.829268, f-Technology=0.798206, pre-Technology=0.89, rec-Technology=0.723577, f-Military=0.84264, pre-Military=0.83, rec-Military=0.85567, f-Government=0.69697, pre-Government=0.69, rec-Government=0.704082, f=0.808, pre=0.808, rec=0.808



2020-07-05 02:55:46 W [callback.py:38] ======epoch : 21 , early stopping : 10/10======
2020-07-05 02:55:46 I [callback.py:40] metric_key : f, metric_value : 0.81
2020-07-05 02:55:46 I [callback.py:41] eval_result : 
{'ClassifyFPreRecMetric': {'f-EnterSports': 0.935961, 'pre-EnterSports': 0.95, 'rec-EnterSports': 0.92233, 'f-Economics': 0.754902, 'pre-Economics': 0.77, 'rec-Economics': 0.740385, 'f-Technology': 0.811881, 'pre-Technology': 0.82, 'rec-Technology': 0.803922, 'f-Military': 0.841026, 'pre-Military': 0.82, 'rec-Military': 0.863158, 'f-Government': 0.704082, 'pre-Government': 0.69, 'rec-Government': 0.71875, 'f': 0.81, 'pre': 0.81, 'rec': 0.81}}
2020-07-05 02:55:46 I [callback.py:46] reach early stopping patience, stop training.
2020-07-05 02:55:46 I [callback.py:55] Early Stopping triggered in epoch 21!


Evaluate data in 0.46 seconds!
Reloaded the best model.

In Epoch:10/Step:790, got best dev performance:
ClassifyFPreRecMetric: f-EnterSports=0.949495, pre-EnterSports=0.94, rec-EnterSports=0.959184, f-Economics=0.762376, pre-Economics=0.77, rec-Economics=0.754902, f-Technology=0.821782, pre-Technology=0.83, rec-Technology=0.813725, f-Military=0.85567, pre-Military=0.83, rec-Military=0.882979, f-Government=0.696078, pre-Government=0.71, rec-Government=0.682692, f=0.816, pre=0.816, rec=0.816


{'best_epoch': 10,
 'best_eval': {'ClassifyFPreRecMetric': {'f': 0.816,
   'f-Economics': 0.762376,
   'f-EnterSports': 0.949495,
   'f-Government': 0.696078,
   'f-Military': 0.85567,
   'f-Technology': 0.821782,
   'pre': 0.816,
   'pre-Economics': 0.77,
   'pre-EnterSports': 0.94,
   'pre-Government': 0.71,
   'pre-Military': 0.83,
   'pre-Technology': 0.83,
   'rec': 0.816,
   'rec-Economics': 0.754902,
   'rec-EnterSports': 0.959184,
   'rec-Government': 0.682692,
   'rec-Military': 0.882979,
   'rec-Technology': 0.813725}},
 'best_step': 790,
 'seconds': 116.97}

模型预测

In [9]:
import torch
import os
import codecs

from fastNLP import Const
from fastNLP import DataSet
from fastNLP.core.predictor import Predictor
from myClue.tools.serialize import load_serialize_obj  # noqa
from myClue.tools.file import read_json_file_iter  # noqa

In [10]:
model_path = '/content/drive/My Drive/game/UCAS_NLP_TC/models/02_train_textcnn_chars'
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
# 获取最新模型文件
model_list = [file_name for file_name in os.listdir(model_path) if 'best_' in file_name]
model_list.sort(key=lambda fn: os.path.getmtime(os.path.join(model_path, fn)) if not os.path.isdir(os.path.join(model_path, fn)) else 0)
model_name = os.path.join(model_path, model_list[-1])
print(model_name)

/content/drive/My Drive/game/UCAS_NLP_TC/models/02_train_textcnn_chars/best_CNNText_f_2020-07-05-02-53-49-113738


In [11]:
logger.warn('开始加载模型')
model = torch.load(model_name)
model.eval()
logger.info('模型加载完毕:\n{}'.format(model))
logger.warn('获取词典')
char_vocab = load_serialize_obj(char_vocab_pkl_file)
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = load_serialize_obj(target_vocab_pkl_file)
logger.info('target_vocab:{}'.format(target_vocab))
logger.warn('加载测试数据')

2020-07-05 02:55:46 W [<ipython-input-11-982aee047e1d>:1] 开始加载模型
2020-07-05 02:55:46 I [<ipython-input-11-982aee047e1d>:4] 模型加载完毕:
CNNText(
  (embed): Embedding(
    (embed): StaticEmbedding(
      (dropout_layer): Dropout(p=0, inplace=False)
      (embedding): Embedding(5027, 100, padding_idx=0)
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(100, 30, kernel_size=(1,), stride=(1,), bias=False)
      (1): Conv1d(100, 40, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): Conv1d(100, 50, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=120, out_features=5, bias=True)
)
2020-07-05 02:55:46 W [<ipython-input-11-982aee047e1d>:5] 获取词典
2020-07-05 02:55:46 I [<ipython-input-11-982aee047e1d>:7] char_vocab:Vocabulary(['近', '日', '，', '胡', '杏']...)
2020-07-05 02:55:46 I [<ipython-input-11-982aee047e1d>:9] target_voca

In [12]:
predict_output_json_file_name = '/content/drive/My Drive/game/UCAS_NLP_TC/pred_data/02_train_textcnn_chars.json'

In [13]:
def news_content_process(news_content):
    """"数据转换处理"""
    news_content = news_content.replace(' ', '')
    return ' '.join(list(news_content))
def _white_space_split(sent):
  return sent.split(' ')

In [14]:
import json
predictor = Predictor(model)
with codecs.open(predict_output_json_file_name, mode='w', encoding='utf8') as fw_json:
  with codecs.open('/content/drive/My Drive/game/UCAS_NLP_TC/data/testdata.txt', mode='r', encoding='utf8') as fr:
    for line in fr:
      line = line.strip()
      if len(line) == 0:
        continue
      label, text = line.split('\t')
      text = news_content_process(text)
      test_data = [_white_space_split(text)]
      dataset = DataSet({Const.INPUT: test_data})
      dataset.add_seq_len(field_name=Const.INPUT)
      dataset.set_input(Const.INPUT, Const.INPUT_LEN)
      char_vocab.index_dataset(dataset, field_name=Const.INPUT)
      batch_output = predictor.predict(data=dataset, seq_len_field_name=Const.INPUT_LEN)
      # logger.info('batch_output : {}'.format(batch_output))
      pred_results = batch_output.get('pred')
      # logger.info('pred results:{}'.format(pred_results[:5]))
      label_id = pred_results[0]
      pred_label = target_vocab.to_word(label_id)
      # 组装成所需格式
      row_data = {}
      row_data['label'] = label
      row_data['pred_label'] = pred_label
      row_data['text'] = text
      fw_json.write('{}\n'.format(json.dumps(row_data, ensure_ascii=False)))

In [15]:
! head -2 /content/drive/My\ Drive/game/UCAS_NLP_TC/pred_data/02_train_textcnn_chars.json

{"label": "EnterSports", "pred_label": "EnterSports", "text": "U 2 3 亚 洲 杯 小 组 赛 第 二 轮 ， 中 国 U 2 3 国 家 队 在 完 全 占 据 场 上 优 势 的 情 况 下 ， 无 奈 久 攻 不 下 ， 0 - 1 不 敌 乌 兹 别 克 队 ， 小 组 赛 前 两 轮 一 胜 一 负 出 场 前 景 成 疑 。 但 虽 败 犹 荣 的 是 ， U 2 3 国 足 在 早 早 丢 球 的 情 况 下 ， 并 没 有 自 乱 阵 脚 ， 在 逆 风 球 中 敢 打 敢 拼 ， 若 不 是 临 门 一 脚 火 候 欠 缺 ， 或 许 至 少 能 拿 到 一 分 。 本 届 U 2 3 亚 洲 杯 ， 在 首 战 对 阵 阿 曼 的 比 赛 中 ， 中 国 队 U 2 3 的 小 伙 子 们 不 但 取 得 了 亚 洲 杯 历 史 首 胜 ， 更 是 表 现 出 难 能 可 贵 的 精 气 神 儿 ， 令 人 耳 目 一 新 。 第 二 轮 对 阵 乌 兹 别 克 斯 坦 ， 中 国 U 2 3 国 家 队 迎 来 真 正 考 验 ， 阿 曼 队 虽 然 是 预 选 赛 力 压 伊 朗 队 的 “ 黑 马 ” ， 但 整 体 实 力 在 亚 洲 难 言 一 流 ， 而 乌 兹 别 克 斯 坦 各 个 级 别 的 国 字 号 球 队 都 是 中 国 队 的 老 对 手 ， 实 力 也 更 胜 一 筹 。 果 然 在 比 赛 开 始 后 仅 仅 第 1 4 分 钟 ， 乌 兹 别 克 斯 坦 队 就 抓 住 一 次 中 国 队 禁 区 解 围 不 当 率 先 取 得 领 先 ， 但 是 瑕 不 掩 瑜 的 是 ， 中 国 队 U 2 3 的 小 伙 子 们 ， 在 踢 逆 风 球 的 情 况 下 ， 并 没 有 自 乱 阵 脚 ， 而 是 展 现 出 强 烈 的 求 胜 欲 望 ， 尤 其 是 下 半 场 ， 比 赛 的 局 面 完 全 被 中 国 队 所 掌 控 ， 就 连 央 视 评 论 员 都 感 叹 这 是 一 场 中 国 队 的 进 攻 演 练 ， 无 论 是 杨 立 瑜 的 包 抄 打 门 ， 还 是 韦 世 豪 的 长 途 奔 袭 后 的 劲 射 ， 都 险 些 洞 穿 对

结果评测

In [16]:
with codecs.open(predict_output_json_file_name, mode='r', encoding='utf8') as fr:
  all_count = 0
  right_count = 0
  for line in fr:
    line = line.strip()
    if len(line) == 0:
      continue
    row_data = json.loads(line)
    all_count += 1
    if row_data['label'] == row_data['pred_label']:
      right_count += 1
  print('acc:{}/{}={}'.format(right_count, all_count, round(right_count/all_count, 4)))

acc:387/500=0.774
